<a href="https://colab.research.google.com/github/JAI0705/Fake-news-classifiaction/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [6]:
# Droping null values
df = df.dropna()

In [7]:
# To get independent features
X = df.drop('label', axis=1)

In [8]:
# To get dependent features
y = df['label']

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.17.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
# Vocabulary size
voc_size = 5000

# One-hot Representation

In [15]:
message = X.copy()

In [16]:
message['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
message

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [19]:
# This is unnessary
message.reset_index(inplace=True)

In [20]:
message

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
# Data preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)



In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [25]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[53, 4369, 1684, 2100, 542, 4476, 661, 3426, 3377, 4407],
 [1761, 490, 2854, 3558, 4496, 579, 4431],
 [1471, 157, 3789, 624],
 [4530, 1405, 871, 549, 4036, 251],
 [4369, 4496, 2663, 4132, 753, 4301, 4496, 811, 3860, 4515],
 [1855,
  4417,
  2479,
  1832,
  3401,
  3463,
  1086,
  4992,
  861,
  1779,
  3635,
  3608,
  1665,
  2385,
  4431],
 [1636, 658, 3905, 2057, 479, 2370, 1348, 2492, 4957, 2770, 4947],
 [271, 1693, 3345, 3320, 521, 3258, 3463, 3526, 4957, 2770, 4947],
 [2265, 1250, 1027, 2804, 954, 4002, 4504, 2743, 3463, 4554],
 [2442, 2569, 3114, 3545, 3119, 4015, 1717, 4332],
 [144, 2031, 271, 1183, 1982, 885, 4167, 1020, 3001, 2281, 2722],
 [549, 4722, 542, 4002, 3463, 521],
 [2644, 2891, 4207, 2277, 4409, 4717, 577, 742, 3178],
 [3314, 1871, 2485, 3387, 3106, 4586, 2106, 4957, 2770, 4947],
 [851, 3305, 4717, 623, 1244, 4957, 2770, 4947],
 [2756, 3909, 1146, 1542, 4910, 4687, 2970, 1310, 4509, 4331],
 [4223, 2805, 490],
 [4051, 4671, 3536, 1143, 3463, 2256, 1223, 4431],
 [174,

# Embedding Representation

In [26]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3426 3377 4407]
 [   0    0    0 ... 4496  579 4431]
 [   0    0    0 ...  157 3789  624]
 ...
 [   0    0    0 ... 4957 2770 4947]
 [   0    0    0 ... 3507 3475 4095]
 [   0    0    0 ... 4411 3068 1697]]


In [27]:
# Creating a model
embedding_vector_features = 40 # feature representation
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [28]:
len(embedded_docs),y.shape

(18285, (18285,))

In [30]:
import numpy as np

In [31]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [32]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

# Model Training

In [35]:
# Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 43ms/step - accuracy: 0.8176 - loss: 0.4286 - val_accuracy: 0.9162 - val_loss: 0.1941
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 52ms/step - accuracy: 0.9427 - loss: 0.1430 - val_accuracy: 0.9175 - val_loss: 0.1948
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.9621 - loss: 0.0987 - val_accuracy: 0.9137 - val_loss: 0.2068
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - accuracy: 0.9779 - loss: 0.0664 - val_accuracy: 0.9167 - val_loss: 0.2192
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 18s 76ms/step - accuracy: 0.9829 - loss: 0.0476 - val_accuracy: 0.9193 - val_loss: 0.3236
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.9924 - loss: 0.0225 - val_accuracy: 0.9125 - val_loss: 0.3612
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - accuracy: 0.9956 - loss: 0.0144 - val_accuracy: 0.9102 - val_loss: 0.4348
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9980 - loss: 0.0076 - va

# Adding Dropout

In [47]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [48]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 53ms/step - accuracy: 0.7735 - loss: 0.4421 - val_accuracy: 0.9158 - val_loss: 0.1969
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 18s 39ms/step - accuracy: 0.9438 - loss: 0.1434 - val_accuracy: 0.9190 - val_loss: 0.1922
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 55ms/step - accuracy: 0.9606 - loss: 0.1056 - val_accuracy: 0.9191 - val_loss: 0.2061
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.9693 - loss: 0.0801 - val_accuracy: 0.9180 - val_loss: 0.2254
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.9770 - loss: 0.0697 - val_accuracy: 0.9171 - val_loss: 0.2962
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - accuracy: 0.9818 - loss: 0.0530 - val_accuracy: 0.9171 - val_loss: 0.2784
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.9877 - loss: 0.0399 - val_accuracy: 0.9109 - val_loss: 0.3140
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - accuracy: 0.9891 - loss: 0.0346 - val

# Preformance Metrics and Accuracy

In [49]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [50]:
y_pred = np.where(y_pred>0.6,1,0)

In [51]:
from sklearn.metrics import confusion_matrix

In [52]:
confusion_matrix(y_test,y_pred)

array([[3090,  329],
       [ 218, 2398]])

In [53]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9093620546810274

In [55]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      3419
           1       0.88      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

